In [1]:
import nltk
import numpy as np
import json
import pickle
from datetime import datetime
from itertools import izip
import random

In [2]:
filepath = '/homes/rpujari/scratch/naacl2019/'

In [3]:
from semeval_pyscripts import text_feature_extraction
from semeval_pyscripts import text_utils
from semeval_pyscripts import amr_utils

In [4]:
def read_data(fp):
    ret_data = []
    d_lines = open(fp, 'r').read().split('\n')
    p_done = []
    q_done = []
    qps = []
    answers = []
    for d_line in d_lines:
        if d_line.strip() != '':
            d_dict = json.loads(d_line)
            p_id, q_id, c_id = (int(x) for x in d_dict['id'].split('_'))
            
            if p_id not in p_done:
                if len(answers) != 0:
                    qps.append((q_doc, answers, 'text'))
                if len(qps) != 0:
                    ret_data.append((p_doc, qps))
                qps = []
                q_done = []
                answers = []
                p_sents = nltk.sent_tokenize(d_dict['d_words'])
                p_doc = []
                for p_sent in p_sents:
                    if type(p_sent) == type('string'):
                        p_doc.append(text_utils.spacy_nlp(unicode(p_sent, errors='ignore')))
                    else:
                        p_doc.append(text_utils.spacy_nlp(p_sent))
                p_done.append(p_id)
            
            if q_id not in q_done:
                if len(answers) != 0:
                    qps.append((q_doc, answers, 'text'))
                answers = []
                q = d_dict['q_words']
                if type(q) == type('string'):
                    q_doc = text_utils.spacy_nlp(unicode(q, errors='ignore'))
                else:
                    q_doc = text_utils.spacy_nlp(q)
                q_done.append(q_id)
            
            a = d_dict['c_words']
            if type(a) == type('string'):
                a_doc = text_utils.spacy_nlp(unicode(a, errors='ignore'))
            else:
                a_doc = text_utils.spacy_nlp(a)
            if d_dict['label'] == 1:
                at = 'True'
            else:
                at = 'False'
            answers.append((a_doc, at, d_dict['id']))
    if len(answers) != 0:
        qps.append((q_doc, answers, 'text'))
    if len(qps) != 0:
        ret_data.append((p_doc, qps))
    return ret_data

In [5]:
multirc_train = read_data(filepath + 'data/multirc/multirc-train-processed.json')
multirc_dev = read_data( filepath + 'data/multirc/multirc-dev-processed.json')
multirc_test = read_data(filepath + 'data/multirc/dev_83-fixedIds-processed.json')
multirc_train_full = json.load(open(filepath + 'data/multirc/train_456-fixedIds.json', 'r'))
multirc_test_orig = json.load(open(filepath + 'data/multirc/dev_83-fixedIds.json', 'r'))
train_paras, dev_paras = pickle.load(open(filepath + 'data/multirc/train_dev_split.pkl'))
multirc_dev_orig = {}
multirc_dev_orig['data'] = [multirc_train_full['data'][x] for x in dev_paras]
multirc_train_orig = {}
multirc_train_orig['data'] = [multirc_train_full['data'][x] for x in train_paras]

Preprocess YF and generate YF results here

In [6]:
def write_results(data, data_orig, yf_res):
    with open(filepath + 'tm_dev_res.json', 'w') as outfile:
        p_num = 0
        dev_res = []
        for para in data:
            q_num = 0
            pid = data_orig['data'][p_num]['id']
            for qp in para[1]:
                q_dict = {}
                q_dict['pid'] = pid
                q_dict['qid'] = str(qp[1][0][2].split('_')[1])
                answers = []
                for ans in qp[1]:
                    answers.append(yf_res[ans[2]])
                q_dict['scores'] = answers
                q_num += 1
                dev_res.append(q_dict)
            p_num += 1
        outfile.write(json.dumps(dev_res))

In [7]:
train_id_dict = {}
yf_train_res = {}
used_qids = {}
qid = 0
cid = 0
with open(filepath + 'results/multirc_train_res', 'r') as if1:
    with open(filepath + 'DRaiL/examples/multirc/train_qids.txt', 'w') as of1:
        with open(filepath + 'DRaiL/examples/multirc/train_cids.txt', 'w') as of2:
            with open(filepath + 'DRaiL/examples/multirc/train_probs.json', 'w') as of3:
                with open(filepath + 'DRaiL/examples/multirc/train_preds.txt', 'w') as of4:
                    with open(filepath + 'DRaiL/examples/multirc/train_in_ques.txt', 'w') as of5:   
                        f_lines = if1.read().split('\n')
                        prob_dict = {}
                        for f_line in f_lines:
                            cols = f_line.split('\t')
                            if len(cols) == 2 and int(cols[0].strip().split('_')[0]) in train_paras:
                                #Find / Assign QID
                                qnum = '_'.join(cols[0].strip().split('_')[:2])
                                if qnum not in used_qids.keys():
                                    used_qids[qnum] = str(qid)
                                    qid1 = str(qid)
                                    qid += 1
                                    of1.write(qid1 + '\n')
                                else:
                                    qid1 = str(used_qids[qnum])
                                    
                                #Assign CID
                                train_id_dict[cols[0].strip()] = cid
                                id1 = str(cid)
                                cid += 1
                                of2.write(id1 + '\n')
                                of5.write(qid1 + '\t' + id1 + '\n')
                                    
                                #Write prediction and probability
                                if float(cols[1].strip()) > 0.5:
                                    yf_train_res[cols[0].strip()] = 1
                                else:
                                    yf_train_res[cols[0].strip()] = 0
                                prob_dict[int(id1)] = float(cols[1].strip())
                                if float(cols[1].strip()) >= 0.5:
                                    of4.write(id1 + '\t1\n')
                                else:
                                    of4.write(id1 + '\t0\n')
                        
                        #Dump prob scores           
                        of3.write(json.dumps(prob_dict))
                        
                        #Dump train answer IDs
                        with open(filepath + 'DRaiL/examples/multirc/train_id_dict.json', 'w') as of6:
                            of6.write(json.dumps(train_id_dict))

In [8]:
dev_id_dict = {}
yf_dev_res = {}
used_qids = {}
qid = 0
cid = 0
with open(filepath + 'results/multirc_train_res', 'r') as if1:
    with open(filepath + 'DRaiL/examples/multirc/dev_qids.txt', 'w') as of1:
        with open(filepath + 'DRaiL/examples/multirc/dev_cids.txt', 'w') as of2:
            with open(filepath + 'DRaiL/examples/multirc/dev_probs.json', 'w') as of3:
                with open(filepath + 'DRaiL/examples/multirc/dev_preds.txt', 'w') as of4:
                    with open(filepath + 'DRaiL/examples/multirc/dev_in_ques.txt', 'w') as of5:   
                        f_lines = if1.read().split('\n')
                        prob_dict = {}
                        for f_line in f_lines:
                            cols = f_line.split('\t')
                            if len(cols) == 2 and int(cols[0].strip().split('_')[0]) in dev_paras:
                                #Find / Assign QID
                                qnum = '_'.join(cols[0].strip().split('_')[:2])
                                if qnum not in used_qids.keys():
                                    used_qids[qnum] = str(qid)
                                    qid1 = str(qid)
                                    qid += 1
                                    of1.write(qid1 + '\n')
                                else:
                                    qid1 = str(used_qids[qnum])
                                    
                                #Assign CID
                                dev_id_dict[cols[0].strip()] = cid
                                id1 = str(cid)
                                cid += 1
                                of2.write(id1 + '\n')
                                of5.write(qid1 + '\t' + id1 + '\n')
                                    
                                #Write prediction and probability
                                if float(cols[1].strip()) > 0.5:
                                    yf_dev_res[cols[0].strip()] = 1
                                else:
                                    yf_dev_res[cols[0].strip()] = 0
                                prob_dict[int(id1)] = float(cols[1].strip())
                                if float(cols[1].strip()) >= 0.5:
                                    of4.write(id1 + '\t1\n')
                                else:
                                    of4.write(id1 + '\t0\n')
                        
                        #Dump prob scores           
                        of3.write(json.dumps(prob_dict))
                        
                        #Dump train answer IDs
                        with open(filepath + 'DRaiL/examples/multirc/dev_id_dict.json', 'w') as of6:
                            of6.write(json.dumps(dev_id_dict))

In [151]:
test_id_dict = {}
yf_test_res = {}
used_qids = {}
qid = 0
cid = 0
with open(filepath + 'results/multirc_test_res', 'r') as if1:
    with open(filepath + 'DRaiL/examples/multirc/test_qids.txt', 'w') as of1:
        with open(filepath + 'DRaiL/examples/multirc/test_cids.txt', 'w') as of2:
            with open(filepath + 'DRaiL/examples/multirc/test_probs.json', 'w') as of3:
                with open(filepath + 'DRaiL/examples/multirc/test_preds.txt', 'w') as of4:
                    with open(filepath + 'DRaiL/examples/multirc/test_in_ques.txt', 'w') as of5:   
                        f_lines = if1.read().split('\n')
                        prob_dict = {}
                        for f_line in f_lines:
                            cols = f_line.split('\t')
                            #print cols
                            if len(cols) == 2 and int(cols[0].strip().split('_')[0]) > -1:
                                #Find / Assign QID
                                qnum = '_'.join(cols[0].strip().split('_')[:2])
                                if qnum not in used_qids.keys():
                                    used_qids[qnum] = str(qid)
                                    qid1 = str(qid)
                                    qid += 1
                                    of1.write(qid1 + '\n')
                                else:
                                    qid1 = str(used_qids[qnum])
                                    
                                #Assign CID
                                test_id_dict[cols[0].strip()] = cid
                                id1 = str(cid)
                                cid += 1
                                of2.write(id1 + '\n')
                                of5.write(qid1 + '\t' + id1 + '\n')
                                    
                                #Write prediction and probability
                                if float(cols[1].strip()) >= 0.5:
                                    yf_test_res[cols[0].strip()] = 1
                                else:
                                    yf_test_res[cols[0].strip()] = 0
                                prob_dict[int(id1)] = float(cols[1].strip())
                                if float(cols[1].strip()) >= 0.5:
                                    of4.write(id1 + '\t1\n')
                                else:
                                    of4.write(id1 + '\t0\n')
                        #Dump prob scores           
                        of3.write(json.dumps(prob_dict))
                        
                        #Dump train answer IDs
                        with open(filepath + 'DRaiL/examples/multirc/test_id_dict.json', 'w') as of6:
                            of6.write(json.dumps(test_id_dict))

with open(filepath + 'DRaiL/examples/multirc/test_pids.txt', 'w') as of0:
    for i, para in enumerate(multirc_test):
        of0.write(str(i) + '\n')
    
with open(filepath + 'DRaiL/examples/multirc/test_in_para.txt', 'w') as of6:
    for key in test_id_dict:
        pid = int(key.split('_')[0])
        of6.write(str(pid) + '\t' + str(test_id_dict[key]) + '\n')
    
with open(filepath + 'DRaiL/examples/multirc/pairs.txt', 'w') as of7:
    for k1 in test_id_dict:
        of7.write(str(test_id_dict[k1]) + '\t' + str(test_id_dict[k1]) + '\n')

In [152]:
write_results(multirc_test, multirc_test_orig, yf_test_res)

In [11]:
import codecs
train_labels = {}
src_writer = codecs.open(filepath + 'nli/preprocess/proc_multirc/src-train.txt', 'w', 'utf-8')
targ_writer = codecs.open(filepath + 'nli/preprocess/proc_multirc/targ-train.txt', 'w', 'utf-8')
ques_writer = codecs.open(filepath + 'nli/preprocess/proc_multirc/ques-train.txt', 'w', 'utf-8')
label_writer = codecs.open(filepath + 'nli/preprocess/proc_multirc/label-train.txt', 'w', 'utf-8')
id_writer = codecs.open(filepath + 'nli/preprocess/proc_multirc/ids-train.txt', 'w', 'utf-8')
for para in multirc_train:
    for ques in para[1]:
        answers = ques[1]
        ques = ' '.join(nltk.word_tokenize(ques[0].text.replace('"', '')))
        for i, a1 in enumerate(answers):
            try:
                hyp_l = a1[1]
                hyp_id = train_id_dict[a1[2]]
                hyp = ' '.join(nltk.word_tokenize(a1[0].text.replace('"', '')))
                train_labels[hyp_id] = hyp_l
                for j, a2 in enumerate(answers):
                    if i != j:
                        prem_l = a2[1]
                        prem_id = train_id_dict[a2[2]]
                        prem = ' '.join(nltk.word_tokenize(a2[0].text.replace('"', '')))
                        src_writer.write(str(hyp_id) + '\t' + hyp + '\n')
                        targ_writer.write(str(prem_id) + '\t' + prem + '\n')
                        ques_writer.write(ques.encode(errors='ignore') + '\n')
                        label_writer.write('neutral\n')
                        id_writer.write(str(hyp_id) + '\t' + str(prem_id) + '\t' + str(hyp_l) + '\t' + str(prem_l) + '\n')
            except KeyError:
                pass
src_writer.close()
targ_writer.close()
ques_writer.close()
label_writer.close()
id_writer.close()
print(len(train_labels))

25818


In [12]:
import codecs
dev_labels = {}
src_writer = codecs.open(filepath + 'nli/preprocess/proc_multirc/src-dev.txt', 'w', 'utf-8')
targ_writer = codecs.open(filepath + 'nli/preprocess/proc_multirc/targ-dev.txt', 'w', 'utf-8')
ques_writer = codecs.open(filepath + 'nli/preprocess/proc_multirc/ques-dev.txt', 'w', 'utf-8')
label_writer = codecs.open(filepath + 'nli/preprocess/proc_multirc/label-dev.txt', 'w', 'utf-8')
id_writer = codecs.open(filepath + 'nli/preprocess/proc_multirc/ids-dev.txt', 'w', 'utf-8')
for para in multirc_dev:
    for ques in para[1]:
        answers = ques[1]
        ques = ' '.join(nltk.word_tokenize(ques[0].text.replace('"', '')))
        for i, a1 in enumerate(answers):
            hyp_id = dev_id_dict[a1[2]]
            hyp_l = a1[1]
            hyp = ' '.join(nltk.word_tokenize(a1[0].text.replace('"', '')))
            dev_labels[hyp_id] = hyp_l
            for j, a2 in enumerate(answers):
                if i != j:
                    prem_id = dev_id_dict[a2[2]]
                    prem_l = a2[1]
                    prem = ' '.join(nltk.word_tokenize(a2[0].text.replace('"', '')))
                    src_writer.write(str(hyp_id) + '\t' + hyp + '\n')
                    targ_writer.write(str(prem_id) + '\t' + prem + '\n')
                    ques_writer.write(ques.encode(errors='ignore') + '\n')
                    label_writer.write('neutral\n')
                    id_writer.write(str(hyp_id) + '\t' + str(prem_id) + '\t' + str(hyp_l) + '\t' + str(prem_l) + '\n')
src_writer.close()
targ_writer.close()
ques_writer.close()
label_writer.close()
id_writer.close()
print(len(dev_labels))

1410


In [13]:
import codecs
test_labels = {}
src_writer = codecs.open(filepath + 'nli/preprocess/proc_multirc/src-test.txt', 'w', 'utf-8')
targ_writer = codecs.open(filepath + 'nli/preprocess/proc_multirc/targ-test.txt', 'w', 'utf-8')
ques_writer = codecs.open(filepath + 'nli/preprocess/proc_multirc/ques-test.txt', 'w', 'utf-8')
label_writer = codecs.open(filepath + 'nli/preprocess/proc_multirc/label-test.txt', 'w', 'utf-8')
id_writer = codecs.open(filepath + 'nli/preprocess/proc_multirc/ids-test.txt', 'w', 'utf-8')
for para in multirc_test:
    for ques in para[1]:
        answers = ques[1]
        ques = ' '.join(nltk.word_tokenize(ques[0].text.replace('"', '')))
        for i, a1 in enumerate(answers):
            hyp_id = test_id_dict[a1[2]]
            hyp_l = a1[1]
            hyp = ' '.join(nltk.word_tokenize(a1[0].text.replace('"', '')))
            test_labels[hyp_id] = hyp_l
            for j, a2 in enumerate(answers):
                if i != j:
                    prem_id = test_id_dict[a2[2]]
                    prem_l = a2[1]
                    prem = ' '.join(nltk.word_tokenize(a2[0].text.replace('"', '')))
                    src_writer.write(str(hyp_id) + '\t' + hyp + '\n')
                    targ_writer.write(str(prem_id) + '\t' + prem + '\n')
                    ques_writer.write(ques.encode(errors='ignore') + '\n')
                    label_writer.write('neutral\n')
                    id_writer.write(str(hyp_id) + '\t' + str(prem_id) + '\t' + str(hyp_l) + '\t' + str(prem_l) + '\n')
src_writer.close()
targ_writer.close()
ques_writer.close()
label_writer.close()
id_writer.close()
print(len(test_labels))

4848


#Preprocess NLI and generate NLI results here

In [45]:
train_edict = {}
for l1 in open(filepath + 'results/multirc_train_nli', 'r').read().split('\n'):
    c1 = l1.split('\t')
    if len(c1) == 4:
        try:
            train_edict[int(c1[0])][int(c1[3])][int(c1[1])] = float(c1[2])
        except KeyError:
            train_edict[int(c1[0])] = ({},{},{})
            train_edict[int(c1[0])][int(c1[3])][int(c1[1])] = float(c1[2])
print len(train_edict)

22265


In [46]:
tot = 0.
corr = 0.
src_writer = open(filepath + 'nli/preprocess/proc_multirc/self_training/src-train.txt', 'w')
targ_writer = open(filepath + 'nli/preprocess/proc_multirc/self_training/targ-train.txt', 'w')
src_ques_writer = open(filepath + 'nli/preprocess/proc_multirc/self_training/src-ques-train.txt', 'w')
targ_ques_writer = open(filepath + 'nli/preprocess/proc_multirc/self_training/targ-ques-train.txt', 'w')
label_writer = open(filepath + 'nli/preprocess/proc_multirc/self_training/label-train.txt', 'w')
joint_writer = open(filepath + 'data/multirc/multirc-joint-train-nli.txt', 'w')
for para in multirc_train:
    for q_num1, ques in enumerate(para[1]):
        answers = ques[1]
        for i, a1 in enumerate(answers):
            try:
                prem_l = a1[1]
                prem_id = train_id_dict[a1[2]]
                prem_id_orig = a1[2]
                for q_num2, ques2 in enumerate(para[1]):
                    answers2 = ques2[1]
                    for j, a2 in enumerate(answers2):
                        if i != j and q_num1 == q_num2:
                            hyp_l = a2[1]
                            hyp_id = train_id_dict[a2[2]]
                            hyp_id_orig = a2[2]

                            #Writing to train file
                            if prem_l == 'True':
                                src_writer.write(unicode.encode(a1[0].text, errors='ignore') + '\n')
                                targ_writer.write(unicode.encode(a2[0].text, errors='ignore') + '\n')
                                src_ques_writer.write(unicode.encode(ques[0].text, errors='ignore') + '\n')
                                targ_ques_writer.write(unicode.encode(ques2[0].text, errors='ignore') + '\n')
                            else:
                                pass
                            if prem_l == 'True' and hyp_l == 'True':
                                label_writer.write('entailment\n')
                                joint_writer.write(prem_id_orig + '\t' + hyp_id_orig + '\t' + 'entailment\n')
                            elif prem_l == 'True' and hyp_l == 'False':
                                label_writer.write('contradiction\n')
                                joint_writer.write(prem_id_orig + '\t' + hyp_id_orig + '\t' + 'contradiction\n')
                            elif prem_l == 'False' and hyp_l == 'False':
                                pass
                                #joint_writer.write(prem_id_orig + '\t' + hyp_id_orig + '\t' + 'neutral\n')
                            elif prem_l == 'False' and hyp_l == 'True':
                                pass
                                #joint_writer.write(prem_id_orig + '\t' + hyp_id_orig + '\t' + 'neutral\n')

                            #Verifying effectiveness
                            if prem_l == 'True':
                                if hyp_id in train_edict[prem_id][0] and hyp_l == 'True':
                                    tot += 1
                                    corr += 1
                                    pass
                                elif hyp_id in train_edict[prem_id][2] and hyp_l == 'False':
                                    tot += 1
                                    corr += 1
                                    pass
                                elif hyp_id in train_edict[prem_id][2] or hyp_id in train_edict[prem_id][2]:
                                    tot += 1
                                    pass
            except KeyError:
                pass
print corr, tot, corr / tot
src_writer.close()
targ_writer.close()
src_ques_writer.close()
targ_ques_writer.close()
label_writer.close()
joint_writer.close()

12845.0 19676.0 0.652825777597


In [47]:
dev_edict = {}
for l1 in open(filepath + 'results/multirc_train_nli', 'r').read().split('\n'):
    c1 = l1.split('\t')
    if len(c1) == 4:
        try:
            dev_edict[int(c1[0])][int(c1[3])][int(c1[1])] = float(c1[2])
        except KeyError:
            dev_edict[int(c1[0])] = ({},{},{})
            dev_edict[int(c1[0])][int(c1[3])][int(c1[1])] = float(c1[2])
print len(dev_edict)

22265


In [48]:
tot = 0.
corr = 0.
src_writer = open(filepath + 'nli/preprocess/proc_multirc/self_training/src-dev.txt', 'w')
targ_writer = open(filepath + 'nli/preprocess/proc_multirc/self_training/targ-dev.txt', 'w')
src_ques_writer = open(filepath + 'nli/preprocess/proc_multirc/self_training/src-ques-dev.txt', 'w')
targ_ques_writer = open(filepath + 'nli/preprocess/proc_multirc/self_training/targ-ques-dev.txt', 'w')
label_writer = open(filepath + 'nli/preprocess/proc_multirc/self_training/label-dev.txt', 'w')
joint_writer = open(filepath + 'data/multirc/multirc-joint-dev-nli.txt', 'w')
for para in multirc_dev:
    for q_num1, ques in enumerate(para[1]):
        answers = ques[1]
        for i, a1 in enumerate(answers):
            try:
                prem_l = a1[1]
                prem_id = dev_id_dict[a1[2]]
                prem_id_orig = a1[2]
                for q_num2, ques2 in enumerate(para[1]):
                    answers2 = ques2[1]
                    for j, a2 in enumerate(answers2):
                        if i != j and q_num1 == q_num2:
                            hyp_l = a2[1]
                            hyp_id = dev_id_dict[a2[2]]
                            hyp_id_orig = a2[2]

                            #Writing to dev file
                            if prem_l == 'True':
                                src_writer.write(unicode.encode(a1[0].text, errors='ignore') + '\n')
                                targ_writer.write(unicode.encode(a2[0].text, errors='ignore') + '\n')
                                src_ques_writer.write(unicode.encode(ques[0].text, errors='ignore') + '\n')
                                targ_ques_writer.write(unicode.encode(ques2[0].text, errors='ignore') + '\n')
                            else:
                                pass
                            if prem_l == 'True' and hyp_l == 'True':
                                label_writer.write('entailment\n')
                                joint_writer.write(prem_id_orig + '\t' + hyp_id_orig + '\t' + 'entailment\n')
                            elif prem_l == 'True' and hyp_l == 'False':
                                label_writer.write('contradiction\n')
                                joint_writer.write(prem_id_orig + '\t' + hyp_id_orig + '\t' + 'contradiction\n')
                            elif prem_l == 'False' and hyp_l == 'False':
                                pass
                                #joint_writer.write(prem_id_orig + '\t' + hyp_id_orig + '\t' + 'neutral\n')
                            elif prem_l == 'False' and hyp_l == 'True':
                                pass
                                #joint_writer.write(prem_id_orig + '\t' + hyp_id_orig + '\t' + 'neutral\n')   

                            #Verifying effectiveness
                            if prem_l == 'True':
                                if hyp_id in dev_edict[prem_id][0] and hyp_l == 'True':
                                    tot += 1
                                    corr += 1
                                    pass
                                elif hyp_id in dev_edict[prem_id][2] and hyp_l == 'False':
                                    tot += 1
                                    corr += 1
                                    pass
                                elif hyp_id in dev_edict[prem_id][0] or hyp_id in dev_edict[prem_id][2]:
                                    tot += 1
                                    pass
            except KeyError:
                pass
print corr, tot, corr / tot
src_writer.close()
targ_writer.close()
src_ques_writer.close()
targ_ques_writer.close()
label_writer.close()
joint_writer.close()

734.0 1227.0 0.598207008965


In [ ]:
with open('/homes/rpujari/scratch/joint_model/results/nli', 'r') as infile:
    with open(filepath + 'nli/preprocess/proc_multirc/multirc_joint_nli', 'w') as outfile:
        for line in infile.read().split('\n'):
            cols = line.split('\t')
            if len(cols) == 4:
                outfile.write(str(test_id_dict[cols[0]]) + '\t' + str(test_id_dict[cols[1]]) + '\t' + cols[3] + '\t' + cols[2] + '\n')

In [153]:
test_edict = {}
for l1 in open(filepath + 'results/multirc_allen_nli', 'r').read().split('\n'):
#for l1 in open(filepath + 'nli/preprocess/proc_multirc/dev_ent_res_st0_0.txt', 'r').read().split('\n'):
    c1 = l1.split('\t')
    if len(c1) == 4:
        try:
            test_edict[int(c1[0])][int(c1[3])][int(c1[1])] = float(c1[2])
        except KeyError:
            test_edict[int(c1[0])] = ({},{},{})
            test_edict[int(c1[0])][int(c1[3])][int(c1[1])] = float(c1[2])
print len(test_edict)

4848


In [154]:
tot = 0.
corr = 0.
src_writer = open(filepath + 'nli/preprocess/proc_multirc/self_training/src-test.txt', 'w')
targ_writer = open(filepath + 'nli/preprocess/proc_multirc/self_training/targ-test.txt', 'w')
src_ques_writer = open(filepath + 'nli/preprocess/proc_multirc/self_training/src-ques-test.txt', 'w')
targ_ques_writer = open(filepath + 'nli/preprocess/proc_multirc/self_training/targ-ques-test.txt', 'w')
label_writer = open(filepath + 'nli/preprocess/proc_multirc/self_training/label-test.txt', 'w')
joint_writer = open(filepath + 'data/multirc/multirc-joint-test-nli.txt', 'w')
for para in multirc_test:
    for q_num1, ques in enumerate(para[1]):
        answers = ques[1]
        for i, a1 in enumerate(answers):
            try:
                prem_l = a1[1]
                prem_id = test_id_dict[a1[2]]
                prem_id_orig = a1[2]
                for q_num2, ques2 in enumerate(para[1]):
                    answers2 = ques2[1]
                    for j, a2 in enumerate(answers2):
                        if i != j or q_num1 != q_num2:
                            hyp_l = a2[1]
                            hyp_id = test_id_dict[a2[2]]
                            hyp_id_orig = a2[2]

                            #Writing to test file
                            if prem_l == 'True':
                                src_writer.write(unicode.encode(a1[0].text, errors='ignore') + '\n')
                                targ_writer.write(unicode.encode(a2[0].text, errors='ignore') + '\n')
                                src_ques_writer.write(unicode.encode(ques[0].text, errors='ignore') + '\n')
                                targ_ques_writer.write(unicode.encode(ques2[0].text, errors='ignore') + '\n')
                            else:
                                pass
                            if prem_l == 'True' and hyp_l == 'True':
                                label_writer.write('entailment\n')
                                joint_writer.write(prem_id_orig + '\t' + hyp_id_orig + '\t' + 'entailment\n')
                            elif prem_l == 'True' and hyp_l == 'False':
                                label_writer.write('contradiction\n')
                                joint_writer.write(prem_id_orig + '\t' + hyp_id_orig + '\t' + 'contradiction\n')
                            elif prem_l == 'False' and hyp_l == 'False':
                                joint_writer.write(prem_id_orig + '\t' + hyp_id_orig + '\t' + 'neutral\n')
                            elif prem_l == 'False' and hyp_l == 'True':
                                joint_writer.write(prem_id_orig + '\t' + hyp_id_orig + '\t' + 'neutral\n')   

                            #Verifying effectiveness
                            if prem_l == 'True':
                                if hyp_id in test_edict[prem_id][0] and test_edict[prem_id][0][hyp_id] > 0.\
                                and hyp_l == 'True':
                                    tot += 1
                                    corr += 1
                                    pass
                                elif hyp_id in test_edict[prem_id][2] and test_edict[prem_id][2][hyp_id] > 0.\
                                and hyp_l == 'False':
                                    tot += 1
                                    corr += 1
                                    pass
                                elif (hyp_id in test_edict[prem_id][0] and test_edict[prem_id][0][hyp_id] > 0.)\
                                or (hyp_id in test_edict[prem_id][2] and test_edict[prem_id][2][hyp_id] > 0.):
                                    tot += 1
                                    pass
            except KeyError:
                pass
print corr, tot, corr / tot
src_writer.close()
targ_writer.close()
src_ques_writer.close()
targ_ques_writer.close()
label_writer.close()
joint_writer.close()

3827.0 6944.0 0.551123271889


In [155]:
ent_dict = {}
contr_dict = {}
with open(filepath + 'DRaiL/examples/multirc/test_entail.txt', 'w') as f1:
    with open(filepath + 'DRaiL/examples/multirc/test_contradict.txt', 'w') as f2:
        for prem_id in test_edict.keys():
            e_d, n_d, c_d = test_edict[prem_id]
            #c_d = test_allen_edict[prem_id][2]
            for hyp_id in e_d.keys():
                if e_d[hyp_id] >= 0.95:
                #and test_labels[prem_id] == 'True' and test_labels[hyp_id] == 'True':
                    f1.write(str(prem_id) + '\t' + str(hyp_id) + '\n')
                    try:
                        ent_dict[prem_id][hyp_id] = e_d[hyp_id]
                    except KeyError:
                        ent_dict[prem_id] = {}
                        ent_dict[prem_id][hyp_id] = e_d[hyp_id]
            for hyp_id in c_d.keys():
                if c_d[hyp_id] >= 0.:
                #and test_labels[prem_id] == 'True' and test_labels[hyp_id] == 'False':
                    f2.write(str(prem_id) + '\t' + str(hyp_id) + '\n')
                    try:
                        contr_dict[prem_id][hyp_id] = c_d[hyp_id]
                    except KeyError:
                        contr_dict[prem_id] = {}
                        contr_dict[prem_id][hyp_id] = c_d[hyp_id]
        print(len(ent_dict), len(contr_dict))
        
        for para in multirc_test:
            for qp in para[1]:
                for i, a1 in enumerate(qp[1]):
                    prem_id = test_id_dict[a1[2]]
                    for j, a2 in enumerate(qp[1]):
                        hyp_id = test_id_dict[a2[2]]
                        if prem_id not in ent_dict.keys():
                            ent_dict[prem_id] = {}
                            if i == j:
                                ent_dict[prem_id][hyp_id] = 1
                            else:
                                ent_dict[prem_id][hyp_id] = 0
                        elif hyp_id not in ent_dict[prem_id].keys():
                            if i == j:
                                ent_dict[prem_id][hyp_id] = 1
                            else:
                                ent_dict[prem_id][hyp_id] = 0
                        if prem_id not in contr_dict.keys():
                            contr_dict[prem_id] = {}
                            contr_dict[prem_id][hyp_id] = 0
                        elif hyp_id not in contr_dict[prem_id].keys():
                            contr_dict[prem_id][hyp_id] = 0
        with open(filepath + 'DRaiL/examples/multirc/test_ent_probs.json', 'w') as f3:
            f3.write(json.dumps(ent_dict))
        with open(filepath + 'DRaiL/examples/multirc/test_contr_probs.json', 'w') as f4:
            f4.write(json.dumps(contr_dict))

(273, 3368)


#Run inference.py on DRail

In [156]:
with open(filepath + 'DRaiL/inferred_res', 'r') as infile:
    lines = infile.read().split('\n')
    res = [x.strip() for x in lines[13][6:-3].split("', '")]

In [157]:
inf_res_list = [0] * 4848
ents = set()
contrs = set()
for r_str in res:
    if r_str.startswith('Has'):
        inf_res_list[int(r_str.split(',')[0][9:])] = int(r_str.split(',')[1][:-1])
    else:
        if r_str.split('(')[0] == 'Entail':
            ents.add(r_str.split('(')[1][:-1])
        elif r_str.split('(')[0] == 'Contradict':
            contrs.add(r_str.split('(')[1][:-1])

print len(ents), len(contrs)
for item in ents:
    if item in contrs:
        print item

5193 6493


In [158]:
gold_res = {}
zeros = 0.
ones = 0.
with open('/homes/rpujari/scratch/raj_qa/results/yf_test_gold.old', 'r') as infile:
    flines = infile.read().split('\n')
    for fline in flines:
        cols = fline.split('\t')
        if len(cols) >= 2:
            gold_res[cols[0].strip()] = int(cols[1])
            if int(cols[1]) == 0:
                zeros += 1
            else:
                ones += 1
print(zeros, ones)

(2773.0, 2075.0)


In [159]:
with open(filepath + 'tm_dev_res.json', 'w') as outfile:
    p_num = 0
    acc = 0.
    test_res = []
    for para in multirc_test:
        q_num = 0
        pid = multirc_test_orig['data'][p_num]['id']
        for qp in para[1]:
            q_dict = {}
            q_dict['pid'] = pid
            q_dict['qid'] = str(qp[1][0][2].split('_')[1])
            answers = []
            for ans in qp[1]:
                answers.append(inf_res_list[test_id_dict[ans[2]]])
                if inf_res_list[test_id_dict[ans[2]]] == gold_res[ans[2]]:
                    acc += 1
            q_dict['scores'] = answers
            q_num += 1
            test_res.append(q_dict)
        p_num += 1
    outfile.write(json.dumps(test_res))
    print(acc / len(inf_res_list))

0.697813531353
